# Leadeship_page_harvesting
   ### 1. Input - company name
   ### 2. searching throught bing for respective ledership page url
   ### 3. Filtering out text and links of each page in a list
   ### 4. Applying it to Gimini-2.0 flash for with a prompt to pull out repective 'Executive_Name', 'Title', 'Company_Name', 'Public_Link' from each url
   ### 5. with 'Executive_Name', 'Title', 'Company_Name' search throught bing to get respective Linkedin url
   ### 6. return 'Executive_Name', 'Title', 'Company_Name', 'Public_Link', 'Linkedin Url' in pandas df for the given company name input.

In [2]:
import random
import string
from bs4 import BeautifulSoup
import requests
from google import genai
import ast
import re
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

class ExecutiveHarvestingOptimized:
    def __init__(self, company_name, api_key="enter your key"):
        self.company_name = company_name
        self.api_key = api_key
        self.headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'accept-language': 'en-US,en;q=0.9,en-IN;q=0.8,ta;q=0.7',
            'avail-dictionary': 'pMvQ2kEO',
            'cache-control': 'max-age=0',
            'ect': '4g',
            'priority': 'u=0, i',
            'sec-ch-ua': '"Not A(Brand";v="8", "Chromium";v="132", "Microsoft Edge";v="132"',
            'sec-ch-ua-arch': '"x86"',
            'sec-ch-ua-bitness': '"64"',
            'sec-ch-ua-full-version': '"132.0.2957.127"',
            'sec-ch-ua-full-version-list': '"Not A(Brand";v="8.0.0.0", "Chromium";v="132.0.6834.111", "Microsoft Edge";v="132.0.2957.127"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-model': '""',
            'sec-ch-ua-platform': '"Windows"',
            'sec-ch-ua-platform-version': '"19.0.0"',
            'sec-fetch-dest': 'document',
            'sec-fetch-mode': 'navigate',
            'sec-fetch-site': 'none',
            'sec-fetch-user': '?1',
            'sec-ms-gec': 'AC81AF6AD88501C2F6376F4159083E316022C9C9C7EC1E66079AEC051A5E002B',
            'sec-ms-gec-version': '2-243.1.3068.127',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36 Edg/132.0.0.0',
            'x-client-data': 'eyIxIjoiMCIsIjIiOiIwIiwiMyI6IjAiLCI0IjoiNjYxNzM1ODE5MTk0MTE3NDc4OSIsIjYiOiJzdGFibGUiLCI5IjoiZGVza3RvcCJ9',
            'x-edge-shopping-flag': '1',
        }
        self.genai_client = genai.Client(api_key=self.api_key)

    def redi_extract_urls(self, url):
        try:
            response = requests.get(url, headers=self.headers, timeout=5)
            response.raise_for_status()
            raw_string = response.text
            url_pattern = r'https?://[^\s]+'
            urls = re.findall(url_pattern, raw_string)
            return urls[0][:-2] if urls else None
        except requests.exceptions.RequestException as e:
            print(f"Error fetching or processing URL: {url} - {e}")
            return None

    def random_generate(self):
        return ''.join(random.choices(string.ascii_letters + string.digits, k=32))

    def _extract_executive_data(self, url):
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'lxml')
            text = soup.get_text()
            cleaned_text = re.sub(r'\s+', ' ', text).strip()
            links = [a['href'] for a in soup.find_all('a', href=True)]
            links_out = []
            for link in links:
              links_out.append(link)

            gemini_prompt = f"{cleaned_text} {links_out} :from the above cleaned text and links from html, plz take Executive_Name, Full_Title, Company_Name, and its Corresponding links(if domain is not there in the link, plz add domain from this link: {url}) in python list of list formet don't give any thing else except python list of list"  # Use url directly here

            gemini_response = self.genai_client.models.generate_content(
                model="gemini-2.0-flash", contents=gemini_prompt
            )
            a = gemini_response.text.replace('python', '').replace('```', '').strip()
            parsed_data = ast.literal_eval(a)
            columns = ['Executive_Name', 'Title', 'Company_Name', 'Public_Link']
            x = pd.DataFrame(parsed_data, columns=columns)
            return x

        except requests.exceptions.RequestException as e:
            print(f"Error fetching URL: {url} - {e}")
            return None
        except Exception as e:
            print(f"Error processing URL {url}: {e}")
            return None

    def leadership_harvesting(self):
        refig = self.random_generate()
        params = {
            'q': self.company_name + " leadership/executive team",
            'form': 'ANNH02',
            'refig': refig,
            'pc': 'HCTS',
        }
        try:
            response = requests.get('https://www.bing.com/search', params=params, headers=self.headers, verify=True, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'lxml')
            div_elements = soup.find_all('div', class_='b_tpcn')
            links = []
            for div in div_elements:
                a_tag = div.find('a')
                if a_tag and 'href' in a_tag.attrs:
                    href_value = a_tag['href']
                    extracted_url = self.redi_extract_urls(url=href_value)
                    if extracted_url:
                        links.append(extracted_url)
        except requests.exceptions.RequestException as e:
            print(f"Error during initial Bing search: {e}")
            return pd.DataFrame(columns=['Executive_Name', 'Title', 'Company_Name', 'Public_Link', 'linkedin_link'])

        try:
            gemini_prompt = f"{links} from the above list of links given, give only official link(offical link means company name in domain place of the url) which related to '{self.company_name} leadership/executive team' and the output only in python list"
            response = self.genai_client.models.generate_content(
                model="gemini-2.0-flash", contents=gemini_prompt
            )
            a = response.text.replace('python', '').replace('```', '').strip()
            out = ast.literal_eval(a)
        except Exception as e:
            print(f"Error using Gemini API to filter links: {e}")
            out = []

        empty_df = pd.DataFrame(columns=['Executive_Name', 'Title', 'Company_Name', 'Public_Link'])


        with ThreadPoolExecutor(max_workers=5) as executor:
            futures = [executor.submit(self._extract_executive_data, url) for url in out]
            for future in as_completed(futures):
                result = future.result()
                if result is not None:
                    empty_df = pd.concat([empty_df, result], ignore_index=True)

        empty_df = empty_df.drop_duplicates(subset='Executive_Name', keep='first').reset_index(drop=True)

        linkedin_link = []
        for i in range(len(empty_df)):
            refig = self.random_generate()
            query = f'''"{empty_df.loc[i, 'Executive_Name']}" + "{empty_df.loc[i, 'Title']}" + "{empty_df.loc[i, 'Company_Name']}" + www.linkedin.com/in'''
            params = {
                'q': query,
                'form': 'ANNH02',
                'refig': refig,
                'pc': 'HCTS',
            }
            try:
                response = requests.get('https://www.bing.com/search', params=params, headers=self.headers, verify=True, timeout=5)
                response.raise_for_status()
                soup = BeautifulSoup(response.text, 'lxml')
                div_element = soup.find_all('div', class_='b_tpcn')
                c = 0
                for j in div_element:
                    href_value = j.find('a')['href']
                    check = self.redi_extract_urls(url=href_value)
                    if check and '.linkedin.' in check and '/company/' not in check and '/jobs/' not in check and '/posts/' not in check and '/today/' not in check and '/pub/' not in check and '/pulse/' not in check:
                        linkedin_link.append(check)
                        c = 1
                        break
            except requests.exceptions.RequestException as e:
                print(f"Error during LinkedIn search for {empty_df.loc[i, 'Executive_Name']}: {e}")
            if c == 0:
                linkedin_link.append('Not Found')

        empty_df["linkedin_link"] = linkedin_link
        return empty_df


company_name = input("Enter the company name: ")
executive_harvester = ExecutiveHarvestingOptimized(company_name)
result_df = executive_harvester.leadership_harvesting()
result_df

Enter the company name: Redmi
Error processing URL https://ir.mi.com/system/files-encrypted/nasdaq_kms/assets/2022/03/11/0-19-46/List%20of%20Directors%20and%20their%20Roles%20and%20Functions.pdf: invalid syntax (<unknown>, line 1)


,Executive_Name,Title,Company_Name,Public_Link,linkedin_link
0,Lei Jun,"Founder, Chairman and CEO",Xiaomi Corporation,https://ir.mi.com/corporate-information/board-...,Not Found
1,Lin Bin,Co-Founder and Vice Chairman,Xiaomi Corporation,https://ir.mi.com/corporate-information/board-...,Not Found
2,Liu De,Co-Founder and Executive Director,Xiaomi Corporation,https://ir.mi.com/corporate-information/board-...,Not Found
3,Liu Qin,Non-Executive Director,Xiaomi Corporation,https://ir.mi.com/corporate-information/board-...,Not Found
4,Chen Dongsheng,Independent Non-Executive Director,Xiaomi Corporation,https://ir.mi.com/corporate-information/board-...,Not Found
5,Wong Shun Tak,Independent Non-Executive Director,Xiaomi Corporation,https://ir.mi.com/corporate-information/board-...,Not Found
6,Cai Jinqing,Independent Non-Executive Director,Xiaomi Corporation,https://ir.mi.com/corporate-information/board-...,https://cn.linkedin.com/in/jinqing-caroline-ca...
7,Lu Weibing,Partner and President,Xiaomi,https://v12.mi.com/bd/about/founder/,Not Found
8,Zeng Xuezhong,Senior Vice President,Xiaomi,https://v12.mi.com/bd/about/founder/,Not Found
9,Yan Kesheng,Vice President,Xiaomi,https://v12.mi.com/bd/about/founder/,Not Found
